# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# !pip install --upgrade 'sqlalchemy<2.0'

In [5]:
%cd '/content/drive/MyDrive/udacitydeeplearning/datascience'

/content/drive/MyDrive/udacitydeeplearning/datascience


In [6]:
# import libraries
import re
import nltk
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con=engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [21]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [22]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(MultinomialNB()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [23]:
# X_train.shape

In [24]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#  Train the pipeline
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ef06bfd3d00>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=MultinomialNB()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [25]:
# Test your model and report classification metrics
Y_pred = pipeline.predict(X_test)


f1_score_result = dict()
# Iterate through each column
for i, column in enumerate(Y.columns):
    f1_score_result[column] = f1_score(Y_test[column], Y_pred[:, i], average='weighted')


In [26]:
print(pd.Series(f1_score_result))

related                   0.692531
request                   0.828704
offer                     0.992569
aid_related               0.751949
medical_help              0.882278
medical_products          0.923919
search_and_rescue         0.963821
security                  0.974899
military                  0.955885
child_alone               1.000000
water                     0.904016
food                      0.839334
shelter                   0.867938
clothing                  0.979168
money                     0.970351
missing_people            0.982870
refugees                  0.951357
death                     0.932731
other_aid                 0.805714
infrastructure_related    0.906998
transport                 0.931886
buildings                 0.924292
electricity               0.965240
tools                     0.990856
hospitals                 0.986861
shops                     0.993712
aid_centers               0.980876
other_infrastructure      0.936484
weather_related     

### 6. Improve your model
Use grid search to find better parameters.

In [ ]:
# Define hyperparameter grid
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],  # Adjust the range of n-grams to consider
    'tfidf__use_idf': [True, False],  # Whether to use inverse document frequency (TF-IDF)
    'clf__estimator__alpha': [0.1, 0.5, 1.0]  # Alpha parameter for Multinomial Naive Bayes
}

# Grid search for hyperparameter tuning
cv = GridSearchCV(pipeline, parameters, cv=3)

cv.fit(X_train, Y_train)

# Get the best parameters
best_params = cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
# Test your model and report classification metrics
Y_pred = cv.predict(X_test)

f1_score_result = dict()
# Iterate through each column
for i, column in enumerate(Y.columns):
    f1_score_result[column] = f1_score(Y_test[column], Y_pred[:, i], average='weighted')

In [29]:
print(pd.Series(f1_score_result))

related                   0.790539
request                   0.886074
offer                     0.992474
aid_related               0.755293
medical_help              0.898877
medical_products          0.934535
search_and_rescue         0.964194
security                  0.974613
military                  0.960784
child_alone               1.000000
water                     0.916012
food                      0.896396
shelter                   0.893686
clothing                  0.983033
money                     0.970526
missing_people            0.982584
refugees                  0.951262
death                     0.940929
other_aid                 0.808738
infrastructure_related    0.907079
transport                 0.933798
buildings                 0.927004
electricity               0.965318
tools                     0.990761
hospitals                 0.986766
shops                     0.993616
aid_centers               0.980781
other_infrastructure      0.936758
weather_related     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [30]:
pipeline_update = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [31]:
#  Train the pipeline
pipeline_update.fit(X_train, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ef06bfd3d00>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [32]:
# Test your model and report classification metrics
Y_pred = pipeline_update.predict(X_test)


f1_score_result_updated = dict()
# Iterate through each column
for i, column in enumerate(Y.columns):
    f1_score_result_updated[column] = f1_score(Y_test[column], Y_pred[:, i], average='weighted')

In [33]:
print(pd.Series(f1_score_result_updated))

related                   0.795710
request                   0.881652
offer                     0.992569
aid_related               0.783106
medical_help              0.895483
medical_products          0.930739
search_and_rescue         0.967448
security                  0.975072
military                  0.960973
child_alone               1.000000
water                     0.948230
food                      0.935667
shelter                   0.929926
clothing                  0.983070
money                     0.973272
missing_people            0.983340
refugees                  0.954343
death                     0.944048
other_aid                 0.811803
infrastructure_related    0.906998
transport                 0.938128
buildings                 0.933187
electricity               0.967090
tools                     0.990856
hospitals                 0.986861
shops                     0.993712
aid_centers               0.980876
other_infrastructure      0.936675
weather_related     

In [ ]:
# Define hyperparameter grid
parameters = {
    'clf__estimator__n_estimators': [100, 200],  # Number of trees (for RandomForest)
    'clf__estimator__max_depth': [10, 20],  # Maximum depth of trees (for RandomForest)
    'clf__estimator__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'clf__estimator__min_samples_leaf': [2, 4],  # Minimum number of samples required to be at a leaf node
}

# Grid search for hyperparameter tuning
model_update = GridSearchCV(pipeline_update, parameters, cv=3)

model_update.fit(X_train, Y_train)

# Get the best parameters
best_params = model_update.best_params_

In [37]:
# Test your model and report classification metrics
Y_pred = model_update.predict(X_test)


f1_score_result_update_model = dict()
# Iterate through each column
for i, column in enumerate(Y.columns):
    f1_score_result_update_model[column] = f1_score(Y_test[column], Y_pred[:, i], average='weighted')

In [38]:
print(pd.Series(f1_score_result_update_model))

related                   0.644142
request                   0.751955
offer                     0.992569
aid_related               0.513744
medical_help              0.880977
medical_products          0.923449
search_and_rescue         0.963821
security                  0.974899
military                  0.955885
child_alone               1.000000
water                     0.904112
food                      0.833218
shelter                   0.867663
clothing                  0.979168
money                     0.970351
missing_people            0.982870
refugees                  0.951357
death                     0.932731
other_aid                 0.805904
infrastructure_related    0.907188
transport                 0.931886
buildings                 0.924292
electricity               0.965240
tools                     0.990856
hospitals                 0.986861
shops                     0.993712
aid_centers               0.980876
other_infrastructure      0.936675
weather_related     

In [41]:
# Combine the Series into a DataFrame
results = {'MultinomialNB_update': f1_score_result,
           'RandomForestClassifier': f1_score_result_updated,
           'RandomForestClassifier_update': f1_score_result_update_model}

df = pd.DataFrame(results)

# Display the resulting DataFrame
df.head(20)

,MultinomialNB_update,RandomForestClassifier,RandomForestClassifier_update
related,0.790539,0.795710,0.644142
request,0.886074,0.881652,0.751955
offer,0.992474,0.992569,0.992569
aid_related,0.755293,0.783106,0.513744
medical_help,0.898877,0.895483,0.880977
medical_products,0.934535,0.930739,0.923449
search_and_rescue,0.964194,0.967448,0.963821
security,0.974613,0.975072,0.974899
military,0.960784,0.960973,0.955885
child_alone,1.000000,1.000000,1.000000


In [42]:
# The model is MultinomialNB_update with best hyper-parameters. We will continue to build our pipeline with this model.

### 9. Export your model as a pickle file

In [ ]:
import pickle
pickle.dump(cv, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.